In [ ]:
pip install transformers[torch]
pip install accelerate -U
pip install torch
pip install datasets

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
import pandas as pd
import numpy as np
import re
import os
import torch
import pickle
import sklearn
import keras
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import AutoModel,AutoTokenizer,AutoModelForSequenceClassification, TrainingArguments, Trainer
from nltk.corpus import stopwords
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
from nltk.stem.snowball import SnowballStemmer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import sent_tokenize, word_tokenize
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoModel, BertModel, AutoTokenizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, accuracy_score, precision_recall_fscore_support

import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
data = load_dataset('csv', data_files={'train': 'df_file.csv'})

# Преобразование меток в числовой формат (если необходимо)
data = data['train'].map(lambda example: {'label': int(example['Label'])})

# Выбор и загрузка модели
model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Токенизация
def tokenize_function(examples):
    return tokenizer(examples['Text'], padding="max_length", truncation=True, return_tensors="pt")

data = data.map(tokenize_function, batched=True)

# Разделение на обучающую и тестовую выборки
train_data, test_data = train_test_split(data, test_size=0.2)

train_data = Dataset.from_dict(train_data)
test_data = Dataset.from_dict(test_data)

# Обучение модели
training_args = TrainingArguments(output_dir="./results", evaluation_strategy="epoch")
trainer = Trainer(model=model, args=training_args, train_dataset=train_data, eval_dataset=test_data)

# Проверка данных перед обучением
print(train_data[0])

# Запуск обучения
trainer.train()

In [ ]:
# получение предсказаний модели
predictions = trainer.predict(test_dataset)
predicted_labels = predictions.predictions.argmax(-1)
true_labels = test_dataset['Label']

accuracy = accuracy_score(true_labels, predicted_labels)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')

# 5. Сохранение модели
trainer.save_model("./bert_res")